In [1]:
#import packages
import numpy as np
import cv2

In [2]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/gopro_overhead_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps

if cap.isOpened() == False:
    print("Cannot open video file.")

In [3]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
print(size)
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1080,1920))

(1080, 1920)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [4]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [5]:
# blob detection
params = cv2.SimpleBlobDetector_Params()
 
#threshold
params.minThreshold = 10;
params.maxThreshold = 200; 
# area parameters
params.filterByArea = True
params.minArea = 100
 
# circular parameters
params.filterByCircularity = True
params.minCircularity = 0.02
#params.maxCircularity = 0.7
 
# convexity parameters
params.filterByConvexity = True
params.minConvexity = 0.02
#params.maxConvexity = 0.7

     
# inertia parameters
params.filterByInertia = True
params.minInertiaRatio = 0.01

#params.maxInertiaRatio = 0.1


 
# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

In [6]:
while True:
    ret, frame = cap.read()
    
    #region of interest
    #height, width, _ = frame.shape
    #print(height, width)
    #roi = frame[130:1800, 200:500]
    
    #object detection
    mask = object_detector.apply(frame)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            #print(x, y)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255,0),3)
            #cv2.ellipse(frame,(x+w, y + h),)
            speed = [x, y]
            # print(speed)
            #text = str(label+1)
            #cv2.putText(frame, text,(20,550),cv2.FONT_HERSHEY_PLAIN,1,(255, 0, 10),3)
    
    #blob detection
    keypoints = detector.detect(frame)
    blank = np.zeros((1, 1))
    blobs = cv2.drawKeypoints(frame, keypoints, blank, (0, 0, 255),
                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    number_of_blobs = len(keypoints)
    text = "Number of Circular Blobs: " + str(len(keypoints))
    cv2.putText(blobs, text, (20, 550),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 2)

    #cv2.imshow("roi", roi)
    cv2.imshow("Frame",frame)
    #cv2.imshow("Mask", mask)
    #cv2.imshow("blobs", blobs)
    output.write(frame)
    key = cv2.waitKey(120)
    if key == 27:
        break
#print(height, width)
cap.release()
output.release()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
